<a href="https://colab.research.google.com/github/AritraStark/Deeplense_GSOC_2022/blob/main/Deeplense_test_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Downloading the data and extracting: 

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')
!tar --extract --file '/content/gdrive/MyDrive/lens_data_alt.tgz' 
print('Extraction done.')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
Extraction done.


Setting up imports

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from sklearn.metrics import roc_auc_score, roc_curve, auc, classification_report
import matplotlib.pyplot as plt
import random
from sklearn.preprocessing import LabelEncoder

In [ ]:
dataset = os.listdir('lens_data')
X = []
y = []
j = 1
for i in d:
    data = np.load('lens_data/' + i, allow_pickle=True)
    img = data[0]
    img = img / 255.0
    img = cv2.resize(img, (64, 64))
    img = np.stack((img, img, img), axis=-1)
    X.append(img)
    y.append(data[1])
    print(j)
    j += 1

X = np.array(X)
y = np.array(y)
X.shape, y.shape

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.imshow(X[i])
    plt.title(y[i])
    plt.axis('off')
plt.show()

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

model.compile(optimizer='adam', loss='mse')

model.summary()

In [ ]:
history = model.fit(train_X, train_y, epochs=10, validation_data=(test_X, test_y))

In [ ]:
mse = model.evaluate(test_X, test_y)

In [ ]:
plt.figure()
plt.title('MSE')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
model.save_weights('test-3-model-CNN.h5')